<a href="https://colab.research.google.com/github/CristhianSeverino/Saas_Sales_Pedictive_Analytics/blob/Optimizaci%C3%B3n/Metricas_SaaS_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.cm as cm
from datetime import datetime

# Configurar estilo de visualizaciones
sns.set(style="whitegrid")
# No usar %matplotlib inline en un script .py, solo en notebooks
# print("=================================   Librerias de Python Ejecutandose Con Exito  ========================================")


In [3]:
def load_and_initial_clean_data(file_path):
    """
    Carga el dataset y realiza una limpieza y preprocesamiento inicial.
    """
    data = pd.read_csv(file_path)
    # print("=================================   Datos Cargados Con Exito  ========================================")
    # print(data.describe())
    # print(data.head())
    # print("\nColumnas",data.columns)

    # print("=================================   Exploracion de nulos y Manejo de duplicados   ========================================")
    # print (" ==========   Valores Nulos   ============")
    # print(data.isnull().sum())
    # print("\nFilas Sin data Faltante ni Nulos:", len(data))

    data = data.drop_duplicates(subset=['Row ID'])
    # print("\nFilas tras eliminar duplicados:", len(data))

    # Convertir Transaction Date a date time
    data['Order Date'] = pd.to_datetime(data['Order Date'])
    # print("\n====================== Fromato de Fecha Aplicado =======================")
    # print(data.head())
    # print("==========================================================================")

    # Rellenar Nulos en sales y profit con 0
    data['Discount'] = data['Discount'].fillna(0)
    # Corrección de inplace=True warnings y asignación directa
    data['Contact Name'] = data['Contact Name'].fillna('Desconocido')
    data['Customer'] = data['Customer'].fillna('Cliente Anónimo')
    data['Sales'] = pd.to_numeric(data['Sales'], errors='coerce').fillna(0)
    data['Profit'] = pd.to_numeric(data['Profit'], errors='coerce').fillna(0)

    # Eliminar filas sin Customer ID u Order Date
    data = data.dropna(subset=['Customer ID', 'Order Date'])
    # print("==========================   Limpieza de Datos, ¡Exitosa! A Celebarlo Con un CAFE   ====================")
    # print("\nFilas tras manejar nulos:", len(data))

    # Estandarizar Segment (Convertir a Mayusculas)
    data['Segment'] = data['Segment'].str.upper()
    # print("====================================================================================================")
    # print("\nValores unicos en Segmet:",data['Segment'].unique())

    # Extraer componentes de fecha
    data['Year'] = data['Order Date'].dt.year
    data['Month'] = data['Order Date'].dt.month
    data['Month_Year'] = data['Order Date'].dt.to_period('M')
    data['Quarter'] = data['Order Date'].dt.quarter
    data['Week'] = data['Order Date'].dt.isocalendar().week.astype(int)

    # Estandarizar columnas de texto
    for col in ['Country', 'City', 'Region', 'Subregion', 'Industry', 'Segment', 'Product']:
        data[col] = data[col].astype(str).str.strip().str.title()

    # Calcular Gross Profit Margin por transacción (necesario para la métrica general y por producto)
    # Manejar división por cero si Sales puede ser 0
    data['Gross Profit Margin'] = (data['Profit'] / data['Sales']) * 100
    data.loc[data['Sales'] == 0, 'Gross Profit Margin'] = 0 # Si las ventas son 0, el margen también es 0

    # Preparación para Cohortes y Churn: Necesitamos la primera fecha de compra para cada cliente.
    customer_first_purchase = data.groupby('Customer ID')['Order Date'].min().reset_index()
    customer_first_purchase.columns = ['Customer ID', 'First Purchase Date']
    data = data.merge(customer_first_purchase, on='Customer ID', how='left')

    # Definir la Cohorte de Adquisición por mes
    data['Acquisition Month'] = data['First Purchase Date'].dt.to_period('M')

    # Crear una Tabla de Cliente-mes para saber si un cliente estuvo activo en un mes dado
    # Esta es 'customer_monthly_activity'
    customer_monthly_activity = data.groupby(['Customer ID', 'Month_Year']).agg(
        Revenue=('Sales', 'sum')
    ).reset_index()

    return data, customer_monthly_activity

In [4]:
# 1. Métricas para el Área de Growth (Crecimiento)
def get_growth_metrics(data):
    """
    Calcula métricas clave para el área de Growth.
    Retorna DataFrames relevantes.
    """
    print("\n--- Calculando Métricas para Growth ---")

    # Clientes Activos Mensuales y su Crecimiento
    customer_base_growth = data.groupby('Month_Year')['Customer ID'].nunique().reset_index()
    customer_base_growth.columns = ['Month_Year', 'Active Customers']
    customer_base_growth['Growth (%)'] = customer_base_growth['Active Customers'].pct_change() * 100
    customer_base_growth['Month_Year'] = customer_base_growth['Month_Year'].dt.to_timestamp() # Para guardado y visualización

    # Nuevos Clientes (Adquisiciones) por Mes
    new_customers_monthly = data.groupby('Acquisition Month')['Customer ID'].nunique().reset_index(name='New_Customers')
    new_customers_monthly['Acquisition Month'] = new_customers_monthly['Acquisition Month'].dt.to_timestamp()

    # Tasa de Órdenes por Cliente (proxy de Conversion Rate)
    orders_per_customer = data.groupby('Customer ID')['Order ID'].nunique().reset_index()
    orders_per_customer.columns = ['Customer ID', 'Total_Orders']

    return {
        'customer_base_growth': customer_base_growth,
        'new_customers_monthly': new_customers_monthly,
        'orders_per_customer': orders_per_customer
    }

In [5]:
# 2. Métricas para el Área de Revenue (Ingresos)
def get_revenue_metrics(data, customer_monthly_activity):
    """
    Calcula métricas clave para el área de Revenue.
    Retorna DataFrames relevantes.
    """
    print("\n--- Calculando Métricas para Revenue ---")
    GROSS_MARGIN = 0.75 # Ejemplo: 75% de margen bruto para una empresa SaaS

    # Monthly Recurring Revenue (MRR)
    mrr_df = data.groupby('Month_Year')['Sales'].sum().reset_index()
    mrr_df.columns = ['Month_Year', 'MRR']
    mrr_df['Month_Year_dt'] = mrr_df['Month_Year'].dt.to_timestamp() # Para visualización

    # Customer Lifetime Value (LTV) Histórico
    customer_revenue = data.groupby('Customer ID')['Sales'].sum().reset_index()
    customer_revenue.columns = ['Customer ID', 'Historical_LTV']

    # Average Revenue Per User (ARPU) por cliente (basado en meses activos)
    customer_arpu = customer_monthly_activity.groupby('Customer ID')['Revenue'].mean().reset_index()
    customer_arpu.columns = ['Customer ID', 'ARPU']

    # Tasa de Churn Mensual Promedio (para LTV Predictivo)
    all_months_for_churn_calc = pd.period_range(start=customer_monthly_activity['Month_Year'].min(),
                                                end=customer_monthly_activity['Month_Year'].max(),
                                                freq='M')
    customer_activity_pivot_for_ltv = customer_monthly_activity.pivot_table(
        index='Customer ID', columns='Month_Year', values='Revenue', aggfunc='sum'
    ).notna().astype(int)
    customer_activity_pivot_for_ltv = customer_activity_pivot_for_ltv.reindex(columns=all_months_for_churn_calc, fill_value=0)

    churn_rates_proportions = []
    for i in range(1, len(customer_activity_pivot_for_ltv.columns)):
        previous_month = customer_activity_pivot_for_ltv.columns[i-1]
        current_month = customer_activity_pivot_for_ltv.columns[i]

        active_prev_month = customer_activity_pivot_for_ltv[customer_activity_pivot_for_ltv[previous_month] == 1]
        churned_this_month = active_prev_month[active_prev_month[current_month] == 0]

        num_churned = len(churned_this_month)
        num_active_prev = len(active_prev_month)

        if num_active_prev > 0:
            churn_rate_prop = (num_churned / num_active_prev)
        else:
            churn_rate_prop = 0
        churn_rates_proportions.append(churn_rate_prop)

    average_monthly_churn_rate = np.mean([rate for rate in churn_rates_proportions if rate > 0])

    if average_monthly_churn_rate == 0:
        average_monthly_churn_rate = 0.0001 # Valor mínimo para evitar LTV infinito

    # LTV Predictivo
    predictive_ltv_per_customer = customer_arpu.copy()
    predictive_ltv_per_customer['Predictive_LTV'] = predictive_ltv_per_customer['ARPU'] * GROSS_MARGIN * (1 / average_monthly_churn_rate)
    predictive_ltv_per_customer['Predictive_LTV'] = predictive_ltv_per_customer['Predictive_LTV'].fillna(0)

    # Unir LTV Histórico y Predictivo para un resumen
    final_ltv_df = pd.merge(customer_revenue, predictive_ltv_per_customer, on='Customer ID', how='left')
    final_ltv_df['Historical_LTV'] = final_ltv_df['Historical_LTV'].fillna(0) # Clientes sin historial de ingresos

    # Gross Profit Margin Global
    overall_gpm = data['Gross Profit Margin'].mean() # Asume que 'Gross Profit Margin' ya está calculado en 'data'

    return {
        'mrr_df': mrr_df,
        'final_ltv_df': final_ltv_df,
        'customer_arpu': customer_arpu,
        'overall_gpm': overall_gpm
    }

In [6]:
# 3. Métricas para el Área de Producto
def get_product_metrics(data):
    """
    Calcula métricas clave para el área de Producto.
    Retorna DataFrames relevantes.
    """
    print("\n--- Calculando Métricas para Producto ---")

    # Gross Profit Margin por Producto
    gpm_by_product = data.groupby('Product')['Gross Profit Margin'].mean().sort_values(ascending=False).reset_index()

    # ARPU por Producto (necesita un cálculo similar a customer_monthly_activity pero por producto)
    product_monthly_activity = data.groupby(['Product', 'Month_Year']).agg(
        TotalRevenue=('Sales', 'sum'),
        UniqueCustomers=('Customer ID', 'nunique')
    ).reset_index()
    product_monthly_activity['ARPU_Product'] = product_monthly_activity.apply(
        lambda row: row['TotalRevenue'] / row['UniqueCustomers'] if row['UniqueCustomers'] > 0 else 0,
        axis=1
    )
    # ARPU promedio de un producto sobre todos los meses
    avg_arpu_by_product = product_monthly_activity.groupby('Product')['ARPU_Product'].mean().reset_index(name='Avg_ARPU_Product')


    return {
        'gpm_by_product': gpm_by_product,
        'avg_arpu_by_product': avg_arpu_by_product
    }


In [7]:
# 4. Métricas para el Área de Customer Experience (CX)
def get_cx_metrics(data, customer_monthly_activity, negative_profit):
    """
    Calcula métricas clave para el área de Customer Experience (CX).
    Retorna DataFrames relevantes.
    """
    print("\n--- Calculando Métricas para CX ---")

    # Tasa de Churn Mensual (usando la lógica de tu script)
    all_months = pd.period_range(start=customer_monthly_activity['Month_Year'].min(),
                                 end=customer_monthly_activity['Month_Year'].max(),
                                 freq='M')
    customer_activity_pivot = customer_monthly_activity.pivot_table(
        index='Customer ID', columns='Month_Year', values='Revenue', aggfunc='sum'
    ).notna().astype(int)
    customer_activity_pivot = customer_activity_pivot.reindex(columns=all_months, fill_value=0)

    churn_rates = []
    for i in range(1, len(customer_activity_pivot.columns)):
        current_month = customer_activity_pivot.columns[i]
        previous_month = customer_activity_pivot.columns[i-1]

        active_prev_month = customer_activity_pivot[customer_activity_pivot[previous_month] == 1]
        churned_this_month = active_prev_month[active_prev_month[current_month] == 0]

        num_churned = len(churned_this_month)
        num_active_prev = len(active_prev_month)

        if num_active_prev > 0:
            churn_rate = (num_churned / num_active_prev) * 100
        else:
            churn_rate = 0
        churn_rates.append({'Month': current_month, 'Churn Rate (%)': churn_rate})

    churn_df = pd.DataFrame(churn_rates)
    if not churn_df.empty:
        churn_df['Month'] = churn_df['Month'].dt.to_timestamp() # Para visualización y guardado

    # Clientes con Profit Negativo (ya lo calculaste, solo lo agrupamos de nuevo para asegurar consistencia)
    negative_by_customer = negative_profit.groupby('Customer ID').agg({
        'Profit': ['count', 'sum', 'mean'],
        'Sales': 'sum',
        'Discount': 'mean'
    }).reset_index()
    negative_by_customer.columns = ['Customer ID', 'Num_Negative_Trans', 'Total_Negative_Profit',
                                    'Avg_Negative_Profit', 'Total_Sales', 'Avg_Discount']

    # Incorporar 'Churn' al pivot de actividad de cliente para el análisis de churn por transacciones negativas
    # Necesitas que churn_df contenga información de churn por Customer ID.
    # El 'churn_df' que creaste es una serie temporal. Necesitamos una asignación de churn por cliente.
    # Para el análisis de churn_by_neg_trans, necesitamos una columna 'Churn' por cliente.
    # Una forma de hacer esto es definir "churned" para cada cliente si no estuvieron activos en el último mes de tus datos.

    # Paso adicional: Crear una columna 'Churn' por Customer ID para el merge
    # Churn: 1 si el cliente no está activo en el último mes de datos, 0 si está activo.
    last_month_in_data = customer_activity_pivot.columns[-1]
    churned_customers_at_end = customer_activity_pivot[customer_activity_pivot[last_month_in_data] == 0].index

    churn_status_by_customer = pd.DataFrame(customer_activity_pivot.index)
    churn_status_by_customer['Churn'] = churn_status_by_customer['Customer ID'].apply(lambda x: 1 if x in churned_customers_at_end else 0)

    # Unir con el estado de churn por cliente
    churn_with_profit = negative_by_customer.merge(
        churn_status_by_customer[['Customer ID', 'Churn']],
        on='Customer ID', how='left'
    ).fillna({'Churn': 0}) # Si un cliente no tuvo transacciones negativas, asumir que no churned para este análisis.

    # Tasa de churn por número de transacciones negativas
    # Filtrar solo si hay 'Num_Negative_Trans' > 0 para evitar grupos de 0s si no tienen sentido para este análisis
    if not churn_with_profit.empty and 'Num_Negative_Trans' in churn_with_profit.columns and 'Churn' in churn_with_profit.columns:
        # Asegúrate de que Num_Negative_Trans sea numérico para el groupby
        churn_with_profit['Num_Negative_Trans'] = pd.to_numeric(churn_with_profit['Num_Negative_Trans'], errors='coerce').fillna(0).astype(int)
        churn_by_neg_trans = churn_with_profit.groupby('Num_Negative_Trans')['Churn'].mean().reset_index()
    else:
        churn_by_neg_trans = pd.DataFrame(columns=['Num_Negative_Trans', 'Churn'])


    # Cohorte de Retención
    # Calculamos el mes de adquisición y luego la actividad en meses subsiguientes.
    data['Months Since Acquisition'] = (data['Month_Year'].apply(lambda x: x.to_timestamp()) -
                                     data['Acquisition Month'].apply(lambda x: x.to_timestamp())).dt.days / 30.4
    data['Months Since Acquisition'] = data['Months Since Acquisition'].astype(int)

    cohort_counts = data.groupby(['Acquisition Month', 'Months Since Acquisition']).agg(
        Customers=('Customer ID', 'nunique')
    ).reset_index()

    cohort_pivot = cohort_counts.pivot_table(
        index='Acquisition Month',
        columns='Months Since Acquisition',
        values='Customers'
    )

    cohort_sizes = cohort_pivot.iloc[:, 0]
    retention_matrix = cohort_pivot.divide(cohort_sizes, axis=0) * 100


    return {
        'churn_rate_monthly_df': churn_df,
        'negative_profit_by_customer': negative_by_customer,
        'churn_by_negative_transactions': churn_by_neg_trans,
        'retention_cohort_matrix': retention_matrix
    }


In [8]:
# --- Función para generar y guardar los CSVs ---
def generate_reports(metrics_dict, output_dir='reports'):
    """
    Genera y guarda los DataFrames de métricas en archivos CSV por área.
    """
    import os
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if not os.path.exists(os.path.join(output_dir, 'plots')):
        os.makedirs(os.path.join(output_dir, 'plots'))

    print(f"\n--- Generando reportes en CSV en la carpeta '{output_dir}' ---")

    # Growth
    growth_metrics = metrics_dict['growth']
    growth_metrics['customer_base_growth'].to_csv(os.path.join(output_dir, 'growth_customer_base_growth.csv'), index=False)
    growth_metrics['new_customers_monthly'].to_csv(os.path.join(output_dir, 'growth_new_customers_monthly.csv'), index=False)
    growth_metrics['orders_per_customer'].to_csv(os.path.join(output_dir, 'growth_orders_per_customer.csv'), index=False)
    print("Reportes de Growth generados.")

    # Revenue
    revenue_metrics = metrics_dict['revenue']
    revenue_metrics['mrr_df'].to_csv(os.path.join(output_dir, 'revenue_mrr_monthly.csv'), index=False)
    revenue_metrics['final_ltv_df'].to_csv(os.path.join(output_dir, 'revenue_ltv_per_customer.csv'), index=False)
    revenue_metrics['customer_arpu'].to_csv(os.path.join(output_dir, 'revenue_arpu_per_customer.csv'), index=False)
    # overall_gpm es un valor escalar, no un DataFrame, puedes guardarlo en un archivo de texto o incluirlo en otro CSV
    with open(os.path.join(output_dir, 'revenue_overall_gpm.txt'), 'w') as f:
        f.write(f"Gross Profit Margin Promedio Global: {revenue_metrics['overall_gpm']:.2f}%\n")
    print("Reportes de Revenue generados.")

    # Product
    product_metrics = metrics_dict['product']
    product_metrics['gpm_by_product'].to_csv(os.path.join(output_dir, 'product_gpm_by_product.csv'), index=False)
    product_metrics['avg_arpu_by_product'].to_csv(os.path.join(output_dir, 'product_avg_arpu_by_product.csv'), index=False)
    print("Reportes de Producto generados.")

    # CX
    cx_metrics = metrics_dict['cx']
    cx_metrics['churn_rate_monthly_df'].to_csv(os.path.join(output_dir, 'cx_monthly_churn_rate.csv'), index=False)
    cx_metrics['negative_profit_by_customer'].to_csv(os.path.join(output_dir, 'cx_customers_with_negative_profit.csv'), index=False)
    cx_metrics['churn_by_negative_transactions'].to_csv(os.path.join(output_dir, 'cx_churn_by_negative_transactions.csv'), index=False)
    # La matriz de retención puede tener un índice complejo, mejor resetearlo para CSV
    cx_metrics['retention_cohort_matrix'].reset_index().to_csv(os.path.join(output_dir, 'cx_retention_cohort_matrix.csv'), index=False)
    print("Reportes de CX generados.")

    print("\nTodos los reportes CSV han sido guardados.")

In [9]:
def generate_visualizations(metrics_dict, output_dir='reports/plots'):
    """
    Genera y guarda visualizaciones clave en archivos de imagen.
    """
    import os
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print(f"\n--- Generando visualizaciones en la carpeta '{output_dir}' ---")

    # --- Growth ---
    growth_metrics = metrics_dict['growth']
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=growth_metrics['customer_base_growth'], x='Month_Year', y='Active Customers', marker='o')
    plt.title('Crecimiento de la Base de Clientes Activos')
    plt.xlabel('Mes')
    plt.ylabel('Número de Clientes Activos')
    plt.xticks(rotation=45)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'growth_customer_base_growth.png'))
    plt.close()

    plt.figure(figsize=(10, 6))
    sns.histplot(growth_metrics['orders_per_customer']['Total_Orders'], bins=20, kde=True)
    plt.title('Distribución de Órdenes por Cliente')
    plt.xlabel('Número Total de Órdenes')
    plt.ylabel('Número de Clientes')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'growth_orders_per_customer_dist.png'))
    plt.close()

    # --- Revenue ---
    revenue_metrics = metrics_dict['revenue']
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=revenue_metrics['mrr_df'], x='Month_Year_dt', y='MRR')
    plt.title('Tendencia de MRR Mensual')
    plt.xlabel('Mes')
    plt.ylabel('MRR')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'revenue_mrr_trend.png'))
    plt.close()

    plt.figure(figsize=(14, 6))
    plt.subplot(1, 2, 1)
    sns.histplot(revenue_metrics['final_ltv_df']['Predictive_LTV'], bins=20, kde=True, color='skyblue')
    plt.title('Distribución del LTV Predictivo por Cliente')
    plt.xlabel('LTV Predictivo ($)')
    plt.ylabel('Número de Clientes')
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.subplot(1, 2, 2)
    sns.histplot(revenue_metrics['final_ltv_df']['Historical_LTV'], bins=20, kde=True, color='lightcoral')
    plt.title('Distribución del LTV Histórico por Cliente')
    plt.xlabel('LTV Histórico ($)')
    plt.ylabel('Número de Clientes')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'revenue_ltv_distributions.png'))
    plt.close()

    # --- Product ---
    product_metrics = metrics_dict['product']
    plt.figure(figsize=(12, 6))
    sns.barplot(data=product_metrics['gpm_by_product'].head(10), x='Product', y='Gross Profit Margin') # Top 10 productos
    plt.title('Margen de Ganancia Bruto Promedio por Producto (Top 10)')
    plt.xlabel('Producto')
    plt.ylabel('Margen de Ganancia Bruto (%)')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'product_gpm_by_product.png'))
    plt.close()

    # --- CX ---
    cx_metrics = metrics_dict['cx']
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=cx_metrics['churn_rate_monthly_df'], x='Month', y='Churn Rate (%)', marker='o')
    plt.title('Tendencia de Tasa de Churn de Clientes (1 mes de inactividad)')
    plt.xlabel('Mes')
    plt.ylabel('Tasa de Churn (%)')
    plt.xticks(rotation=45)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'cx_monthly_churn_rate_trend.png'))
    plt.close()

    plt.figure(figsize=(14, 8))
    sns.heatmap(cx_metrics['retention_cohort_matrix'], annot=True, fmt=".1f", cmap="Blues",
                cbar_kws={'label': 'Retención (%)'})
    plt.title('Matriz de Retención por Cohorte de Adquisición')
    plt.xlabel('Meses Desde la Adquisición')
    plt.ylabel('Mes de Adquisición')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'cx_retention_cohort_matrix.png'))
    plt.close()

    if not cx_metrics['churn_by_negative_transactions'].empty:
        plt.figure(figsize=(10, 6))
        sns.barplot(data=cx_metrics['churn_by_negative_transactions'], x='Num_Negative_Trans', y='Churn')
        plt.title('Tasa de Churn por Número de Transacciones con Profit Negativo')
        plt.xlabel('Número de Transacciones Negativas')
        plt.ylabel('Tasa de Churn')
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'cx_churn_by_negative_transactions.png'))
        plt.close()
    else:
        print("No se generó gráfico de Churn por Transacciones Negativas: DataFrame vacío o columnas faltantes.")


    print("Todas las visualizaciones han sido guardadas.")

In [10]:
# --- Función Principal para ejecutar todo el flujo ---
def main_analysis(file_path='SaaS-Sales.csv'):
    """
    Función principal que orquesta todo el análisis: carga, limpieza,
    cálculo de métricas por área y generación de reportes/visualizaciones.
    """
    print("Iniciando análisis de métricas SaaS...")

    # 1. Cargar y limpiar datos iniciales
    data, customer_monthly_activity = load_and_initial_clean_data(file_path)
    print("\nDatos cargados y preprocesados.")

    # 2. Calcular métricas específicas para el análisis de profit negativo
    # Esto es necesario porque `get_cx_metrics` lo utiliza
    negative_profit = data[data['Profit'] < 0]

    # 3. Calcular métricas para cada área
    growth_metrics = get_growth_metrics(data)
    revenue_metrics = get_revenue_metrics(data, customer_monthly_activity)
    product_metrics = get_product_metrics(data)
    cx_metrics = get_cx_metrics(data, customer_monthly_activity, negative_profit)

    # Consolidar todas las métricas en un solo diccionario
    all_metrics = {
        'growth': growth_metrics,
        'revenue': revenue_metrics,
        'product': product_metrics,
        'cx': cx_metrics
    }
    print("\nMétricas calculadas para todas las áreas.")

    # 4. Generar reportes CSV
    generate_reports(all_metrics)

    # 5. Generar visualizaciones
    generate_visualizations(all_metrics)

    print("\nAnálisis completado. Revise la carpeta 'reports' para los resultados.")

# --- Ejecutar el análisis principal ---
if __name__ == "__main__":
    main_analysis()


Iniciando análisis de métricas SaaS...

Datos cargados y preprocesados.

--- Calculando Métricas para Growth ---

--- Calculando Métricas para Revenue ---

--- Calculando Métricas para Producto ---

--- Calculando Métricas para CX ---

Métricas calculadas para todas las áreas.

--- Generando reportes en CSV en la carpeta 'reports' ---
Reportes de Growth generados.
Reportes de Revenue generados.
Reportes de Producto generados.
Reportes de CX generados.

Todos los reportes CSV han sido guardados.

--- Generando visualizaciones en la carpeta 'reports/plots' ---
Todas las visualizaciones han sido guardadas.

Análisis completado. Revise la carpeta 'reports' para los resultados.
